### Install packages

In [1]:
! pip install -r requirements.txt --quiet

### Load .env file

Set the appropriate environment variables below:



In [2]:
from dotenv import load_dotenv
import os

load_dotenv(override=True) # take environment variables from .env.


azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
azure_openai_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]


## Write output to a CSV file

In [ ]:
import csv
def write_markdown_table_to_csv(markdown_content, output_file):
    lines = [line.strip() for line in markdown_content.strip().split('\n') if line.strip()]
    # Filter out separator lines (those that start with |---)
    data_lines = [line for line in lines if not set(line.replace('|', '').strip()) <= set('-:')]
    rows = [ [cell.strip() for cell in line.strip('|').split('|')] for line in data_lines ]
    with open(output_file, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

## Use Azure OpenAI to analyze sentiment & pain points in a survey

In [ ]:

from openai import AzureOpenAI
from openai.types.chat import ChatCompletion, ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam, ChatCompletionMessage, ChatCompletionMessageParam
from azure.identity.aio import DefaultAzureCredential, get_bearer_token_provider
import pandas as pd


token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)

def process_csv_files(folder_path):
    """
    Process all CSV files in the specified folder.
    Args:
        folder_path (str): The path to the folder containing CSV files.
        output_folder (str): The path to the folder where output CSVs will be saved.
 
    """
    assistant_system_message = """
Put the system prompt here. This is a template for the assistant to follow when processing the CSV files.

"""
    query = """
Put the query here. This is the question or instruction you want to ask the assistant regarding the CSV data.
"""

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {file_path}")

            df = pd.read_csv(file_path)
            csv_content = df.to_csv(index=False)

            query_message = f"{query}\nRelevant data: {csv_content}"
            messages =   [
            ChatCompletionSystemMessageParam(role="system", content=assistant_system_message),
            ChatCompletionUserMessageParam(role="user", content=query_message),
            ] 
        
            response: ChatCompletion = client.chat.completions.create(
            model=azure_openai_deployment_name,
                messages=messages,
                max_tokens=15000,
                temperature=0.7,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None,
                stream=False
            )

            message: ChatCompletionMessage = response.choices[0].message
            print(f"{message.content}")


            output_folder= r"YOur path to the output folder"
            # Try to parse the output as CSV and save it
            output_file = os.path.join(output_folder, f"output_{filename}")
            write_markdown_table_to_csv(message.content, output_file)

            print(f"Saved output to {output_file}")


process_csv_files(r"Your path to the input folder")